In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
pd.set_option('display.max_columns', 100)

data_2021 = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv', low_memory = False, encoding='UTF-8')
questions_2021 = data_2021.iloc[0, :].T
data_2021 = data_2021.iloc[1:, :]
data_2020 = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv', low_memory = False, encoding='UTF-8')
questions_2020 = data_2020.iloc[0, :].T
data_2020 = data_2020.iloc[1:, :]
data_2019 = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv', low_memory = False, encoding='UTF-8')
questions_2019 = data_2019.iloc[0, :].T
data_2019 = data_2019.iloc[1:, :]
data_2018 = pd.read_csv('../input/kaggle-survey-2018/multipleChoiceResponses.csv', low_memory = False, encoding='UTF-8')
questions_2018 = data_2018.iloc[0, :].T
data_2018 = data_2018.iloc[1:, :]
data_2017 = pd.read_csv('../input/kaggle-survey-2017/multipleChoiceResponses.csv', low_memory = False, encoding='ISO-8859-1')
questions_2017 = data_2017.iloc[0, :].T
data_2017 = data_2017.iloc[1:, :]

# 2021 Yes. 2020 Yes. 2019 Yes. About TPUs

In [5]:
def get_professionals(data, column):
    data = data.loc[data[column] != 'Student']
    data = data.loc[data[column] != 'Currently not employed']
    data = data.loc[data[column] != 'Not employed']
    data = data.loc[data[column].notna()]
    return data

How data science fields has evolved from the 2018 to 2021? 

In [2]:
# Exhibit 1. Data Science Professionals distribution by industry 2018 vs. 2021

# Get data from 2021

industry_2021 = data_2021[data_2021['Q20'].notna()]
c = industry_2021['Q20'].value_counts(normalize=True).rename_axis('industry').reset_index(name='counts')
#c =industry_2021['Q20'].value_counts().rename_axis('industry').reset_index(name='counts')

# Get data from 2018

industry_2018 = data_2018[data_2018['Q7'] != 'I am a student']
industry_2018 = industry_2018[industry_2018['Q7'].notna()]
d = industry_2018['Q7'].value_counts(normalize=True).rename_axis('industry').reset_index(name='counts')
#d = industry_2018['Q7'].value_counts().rename_axis('industry').reset_index(name='counts')

# compute the industry

k = pd.merge(left = d, right = c, on = 'industry')
k = k.rename(columns = {'counts_x': '2018', 'counts_y': '2021'})
k = k.sort_values(by=['2021'], ascending=False)

# compute the difference
diff_industry = k.copy()
diff_industry['dff'] = k['2021'] - k['2018']

# plot

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize = (16,8))
#plt.style.use('IPython_default')
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
ax1 = plt.subplot(gs[0])

k.plot.barh(x = "industry", ax= ax1)
#ax.grid(False)
ax1.set(title = "Data Science professionals distribution by Industry. 2018 vs 2021",
      xlabel = "Percentage",
      ylabel = "Industry")
ax1.invert_yaxis()

ax2 = plt.subplot(gs[1])
diff_industry['dff'].plot(kind='barh', x = 'industry', ax = ax2,
                    color=(diff_industry['dff'] > 0).map({True: 'g',
                                                    False: 'r'}))
ax2.set(title = "Change",
      xlabel = "Percentage",
      ylabel = "Industry")
ax2.set_yticks([])

plt.gca().invert_yaxis()

* Computer/Technology field is no longer the only game in town.
* Academic/Education industry and Manufactruing industry experienced siginificant jump.

2017, 2019, 2020 Do not have industry field data

In [4]:
# Exhibit 1.1 Data Science distribution by company size 2021.

test = get_professionals(data_2021, 'Q5')
#print(len(test))
test = test[test['Q21'].notna()]
#print(len(test))

category_test = test.groupby(['Q20', 'Q21']).size()
#category_test.plot(kind='bar')
new_df = category_test.to_frame(name = 'size').reset_index()
new_df_2= pd.pivot(
    data = new_df,
    index = 'Q20',
    columns = 'Q21',
    values = 'size')
new_df_2.index.names = ['Industry']
new_df_2.columns.names = ['Company Size']

columns_order = ['0-49 employees', '50-249 employees', '250-999 employees', '1000-9,999 employees','10,000 or more employees']

new_df_2 = new_df_2.reindex(columns = columns_order)
new_df_2['total'] = new_df_2[columns_order].sum(axis = 1)
new_df_2 = new_df_2.sort_values(by = 'total', ascending = False)
new_df_2 = new_df_2.drop(columns='total')

# -----


new_df_2['total'] = new_df_2[columns_order].sum(axis = 1)
new_df_2 = new_df_2.sort_values(by = 'total', ascending = False)
new_df_2 = new_df_2.drop(columns = 'total')
res = new_df_2.div(new_df_2.sum(axis=1), axis = 0)
res

# -----

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize = (16,8))
#plt.style.use('IPython_default')
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 

ax1 = plt.subplot(gs[0])

new_df_2.plot(use_index = True,  
              kind='barh', 
              stacked=True, 
              ax = ax1,
              )

ax1.set(title = "DS professional distibution by company size across different industry 2021",
      xlabel = "Counts",
      ylabel = "Industry")


ax2 = plt.subplot(gs[1])

res.plot(use_index = True,  
              kind='barh', 
              stacked=True, 
              ax = ax2,
              )

ax2.set(title='Company Size portion within industry',
      xlabel = "Percentage",
      ylabel = " ")

plt.legend(title = "Company Size", bbox_to_anchor=(1.04,1), loc="upper left")
ax2.set_yticks([])
plt.show()

(0-49) sized company accounts the most except insurance/Risk industry.


In [5]:
# 2021, 2020, 2019 (#26 and #27) About Computer vision and NLP

In [128]:
final_boss

In [266]:
# COMPUTER VISION YES OR NO in 2021

#data_2021.iloc[:,85:120].head()
test = get_professionals(data_2021, 'Q5')
vision = test[['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_Part_6','Q18_OTHER']]
nlp = test[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER']]
vision = vision.fillna(0)
vision[vision != 0] = 1
vision_df = vision[vision==True].count(axis=0).rename_axis('Algo').reset_index(name='counts')
vision_df = vision_df.set_index('Algo').T
stuff_2020 = vision_df

vision['yes'] = vision[['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_OTHER']].sum(axis = 1)
vision['yes'] = vision['yes'].apply(lambda x: x>=1)
#vision['yes'].value_counts()
vision['no'] = (vision['yes'].apply(lambda x: x == 0) | vision['Q18_Part_6'].apply(lambda x: x == 1))
#vision['no'].value_counts()

vision['Q20'] = test['Q20']
vision = vision.drop(columns=['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_Part_6','Q18_OTHER'])
vision.replace({False: 0, True: 1}, inplace=True)
vision_df = vision[vision['yes'] == 1].groupby('Q20').size()

total_boss = vision['Q20'].value_counts()

boss = pd.DataFrame(vision_df)
total_boss = pd.DataFrame(total_boss)
final_boss = total_boss.join(boss)
final_boss.rename(columns = {final_boss.columns[0]: 'NO', final_boss.columns[1]: 'YES'}, inplace = True)
#final_boss

# NLP YES OR NO in 2021

nlp = test[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER']]
nlp = nlp.fillna(0)
nlp[nlp != 0] = 1
nlp_df = nlp[nlp==True].count(axis=0).rename_axis('Algo').reset_index(name='counts')
nlp_df = nlp_df.set_index('Algo').T

nlp['yes'] = nlp[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_OTHER']].sum(axis = 1)
nlp['yes'] = nlp['yes'].apply(lambda x: x>=1)
#vision['yes'].value_counts()
nlp['no'] = (nlp['yes'].apply(lambda x: x == 0) | nlp['Q19_Part_5'].apply(lambda x: x == 1))
#vision['no'].value_counts()

nlp['Q20'] = test['Q20']
nlp = nlp.drop(columns=['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER'])
nlp.replace({False: 0, True: 1}, inplace=True)
nlp_df = nlp[nlp['yes'] == 1].groupby('Q20').size()

total_boss2 = nlp['Q20'].value_counts()

boss2 = pd.DataFrame(nlp_df)
total_boss2 = pd.DataFrame(total_boss2)
final_boss2 = total_boss.join(boss2)
final_boss2.rename(columns = {final_boss2.columns[1]: 'hello'}, inplace = True)
#final_boss2['perc'] = final_boss2['hello'] * 100 / final_boss2['Q20']
final_boss2.rename(columns = {final_boss2.columns[0]: 'NO', final_boss2.columns[1]: 'YES'}, inplace = True)
final_boss2

fig, ax1 = plt.subplots(figsize = (12,11))

final_boss.plot(kind='barh', ax = ax1)
ax1.set(title = "Computer Vision Yes / No")

for i,j in zip(ax1.containers[0], ax1.containers[1]):

    perc = j.get_width() / i.get_width()
    perc = (perc*100).round(2)
    non_perc = (100 - perc).round(2)
    
    width = i.get_width()
    height = i.get_height()
    x, y = i.get_xy()
    ax1.annotate(f'{non_perc}%', (x + width, y + height*1.02), ha="left", va="center")
    
    width2 = j.get_width()
    height2 = j.get_height()
    x2, y2 = j.get_xy() 
    ax1.annotate(f'{perc}%', (x + width2, y2 + height2*1.02), ha="left", va="center")

In [267]:
nlp = test[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER']]
nlp = nlp.fillna(0)
nlp[nlp != 0] = 1
nlp_df = nlp[nlp==True].count(axis=0).rename_axis('Algo').reset_index(name='counts')
nlp_df = nlp_df.set_index('Algo').T

nlp['yes'] = nlp[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_OTHER']].sum(axis = 1)
nlp['yes'] = nlp['yes'].apply(lambda x: x>=1)
#vision['yes'].value_counts()
nlp['no'] = (nlp['yes'].apply(lambda x: x == 0) | nlp['Q19_Part_5'].apply(lambda x: x == 1))
#vision['no'].value_counts()

nlp['Q20'] = test['Q20']
nlp = nlp.drop(columns=['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER'])
nlp.replace({False: 0, True: 1}, inplace=True)
nlp_df = nlp[nlp['yes'] == 1].groupby('Q20').size()

total_boss2 = nlp['Q20'].value_counts()

boss2 = pd.DataFrame(nlp_df)
total_boss2 = pd.DataFrame(total_boss2)
final_boss2 = total_boss.join(boss2)
final_boss2.rename(columns = {final_boss2.columns[1]: 'hello'}, inplace = True)
#final_boss2['perc'] = final_boss2['hello'] * 100 / final_boss2['Q20']
final_boss2.rename(columns = {final_boss2.columns[0]: 'NO', final_boss2.columns[1]: 'YES'}, inplace = True)
final_boss2

fig, ax2 = plt.subplots(figsize = (12,11))
final_boss2.plot(kind='barh', ax = ax2)
ax2.set(title = "NLP Yes / No")

for i,j in zip(ax2.containers[0], ax2.containers[1]):

    perc = j.get_width() / i.get_width()
    perc = (perc*100).round(2)
    non_perc = (100 - perc).round(2)
    
    width = i.get_width()
    height = i.get_height()
    x, y = i.get_xy()
    ax2.annotate(f'{non_perc}%', (x + width, y + height*1.02), ha="left", va="center")
    
    width2 = j.get_width()
    height2 = j.get_height()
    x2, y2 = j.get_xy() 
    ax2.annotate(f'{perc}%', (x + width2, y2 + height2*1.02), ha="left", va="center")
    
plt.show()

In [95]:
algo_set = stuff_2021.append([stuff_2020, stuff_2019])
algo_set


In [96]:
algo_set = algo_set.T
fig, ax1 = plt.subplots(figsize = (16,8))
algo_set.plot(kind = 'bar', ax = ax1)
ax1.set(title = "ML Algo")

In [89]:
#data_2021.iloc[:,85:120].head()

In [81]:
#data_2021.iloc[:,85:120].head()
test = get_professionals(data_2021, 'Q5')

old_colnames = ['Q17_Part_1', 'Q17_Part_2', 'Q17_Part_3', 'Q17_Part_4', 'Q17_Part_5', 'Q17_Part_6', 'Q17_Part_7', 'Q17_Part_8', 'Q17_Part_9','Q17_Part_10', 'Q17_Part_11', 'Q17_OTHER']
new_colnames = ['Regression', 'Decision Trees', 'Gradient Boosting', 'Bayesian', 'Evolutionary', 'DNN', 'CNN', 'GAN', 'RNN', 'Transformer', 'None', 'Other']


algos = test[['Q17_Part_1', 'Q17_Part_2', 'Q17_Part_3', 'Q17_Part_4', 'Q17_Part_5', 'Q17_Part_6', 'Q17_Part_7', 'Q17_Part_8', 'Q17_Part_9','Q17_Part_10', 'Q17_Part_11', 'Q17_OTHER']]
col_rename_dict = {i:j for i,j in zip(old_colnames,new_colnames)}
algos.rename(columns=col_rename_dict, inplace=True)

algos = algos.fillna(0)
algos[algos != 0] = 1
algos.replace({False: 0, True: 1}, inplace=True)
algo_df = algos[algos==True].count(axis=0).rename_axis('Algo').reset_index(name='2021')
algo_df = algo_df.set_index('Algo').T
stuff_2021 = algo_df
algo_df.plot(kind='bar')

In [82]:
#data_2021.iloc[:,85:120].head()
test = get_professionals(data_2020, 'Q5')

old_colnames = ['Q17_Part_1', 'Q17_Part_2', 'Q17_Part_3', 'Q17_Part_4', 'Q17_Part_5', 'Q17_Part_6', 'Q17_Part_7', 'Q17_Part_8', 'Q17_Part_9','Q17_Part_10', 'Q17_Part_11','Q17_OTHER']
new_colnames = ['Regression', 'Decision Trees', 'Gradient Boosting', 'Bayesian', 'Evolutionary', 'DNN', 'CNN', 'GAN', 'RNN', 'Transformer', 'None', 'Other']


algos = test[['Q17_Part_1', 'Q17_Part_2', 'Q17_Part_3', 'Q17_Part_4', 'Q17_Part_5', 'Q17_Part_6', 'Q17_Part_7', 'Q17_Part_8', 'Q17_Part_9','Q17_Part_10', 'Q17_Part_11','Q17_OTHER']]
col_rename_dict = {i:j for i,j in zip(old_colnames,new_colnames)}
algos.rename(columns=col_rename_dict, inplace=True)

algos = algos.fillna(0)
algos[algos != 0] = 1
algos.replace({False: 0, True: 1}, inplace=True)
algo_df = algos[algos==True].count(axis=0).rename_axis('Algo').reset_index(name='2020')
algo_df = algo_df.set_index('Algo').T
stuff_2020 = algo_df
algo_df.plot(kind='bar')

In [87]:
#data_2019.iloc[:,100:150].head(50)

In [88]:
old_colnames = ['Q24_Part_1','Q24_Part_2','Q24_Part_3','Q24_Part_4','Q24_Part_5','Q24_Part_6','Q24_Part_7','Q24_Part_8','Q24_Part_9','Q24_Part_10','Q24_Part_11', 'Q24_Part_12']
#new_colnames = ['Q17_Part_1', 'Q17_Part_2', 'Q17_Part_3', 'Q17_Part_4', 'Q17_Part_5', 'Q17_Part_6', 'Q17_Part_7', 'Q17_Part_8', 'Q17_Part_9','Q17_Part_10', 'Q17_Part_11']

new_colnames = ['Regression', 'Decision Trees', 'Gradient Boosting', 'Bayesian', 'Evolutionary', 'DNN', 'CNN', 'GAN', 'RNN', 'Transformer', 'None', 'Other']
col_rename_dict = {i:j for i,j in zip(old_colnames,new_colnames)}


pd.set_option('display.max_columns', None)
data_2019.head()
algos = data_2019[['Q24_Part_1','Q24_Part_2','Q24_Part_3','Q24_Part_4','Q24_Part_5','Q24_Part_6','Q24_Part_7','Q24_Part_8','Q24_Part_9','Q24_Part_10','Q24_Part_11','Q24_Part_12']]
col_rename_dict = {i:j for i,j in zip(old_colnames,new_colnames)}
algos.rename(columns=col_rename_dict, inplace=True)

algos = algos.fillna(0)
algos[algos != 0] = 1
algos.replace({False: 0, True: 1}, inplace=True)
algo_df = algos[algos==True].count(axis=0).rename_axis('Algo').reset_index(name='2019')
algo_df = algo_df.set_index('Algo').T
stuff_2019 = algo_df
algo_df.plot(kind='bar')

In [2]:
student_2021 = data_2021.loc[data_2021['Q5'] == 'Student']
workforce_2021 = data_2021.loc[data_2021['Q5'] != 'Student']

professional_2021 = workforce_2021[workforce_2021['Q20'].notna()]

industry_2021 = professional_2021['Q20'].unique()

df_2021 = professional_2021[['Q5','Q20']]

d_2021 = {}

for industry in industry_2021:
    temp_df = df_2021[df_2021['Q20'] == industry]
    temp_dict = dict(temp_df['Q5'].value_counts())
    d_2021[industry] = temp_dict

def sorted_simple_dict(d):
    return {k: v for k, v in sorted(d.items())}

def sorted_once_nested_dict(d):
    return {k: sorted_simple_dict(v) for k, v in sorted(d.items())}

d_2021 = sorted_once_nested_dict(d_2021)

df_industry_2021 = pd.DataFrame.from_dict(d_2021, orient='index')
df_industry_2021.fillna(0, inplace = True)

# /-------------------------------------------------------------------------/
# /-------------------------------------------------------------------------/
# /-------------------------------------------------------------------------/

student_2018 = data_2018.loc[data_2018['Q7'] == 'I am a student']
workforce_2018 = data_2018.loc[data_2018['Q7'] != 'I am a student']

professional_2018 = workforce_2018[workforce_2018['Q7'].notna()]
professional_2018.head()

industry_2018 = professional_2018['Q7'].unique()

df_2018 = professional_2018[['Q6','Q7']]

d_2018 = {}

for industry in industry_2018:
    temp_df = df_2018[df_2018['Q7'] == industry]
    temp_dict = dict(temp_df['Q6'].value_counts())
    d_2018[industry] = temp_dict

d_2018 = sorted_once_nested_dict(d_2018)
    
df_industry_2018 = pd.DataFrame.from_dict(d_2018, orient='index')
df_industry_2018.fillna(0, inplace = True)

# /-------------------------------------------------------------------------/
# /-------------------------------------------------------------------------/
# /-------------------------------------------------------------------------/

#  SUBPLOTS - 1x2
fig = plt.figure(figsize=(22,10))

plt.subplot(121)   #  subplot 1
plt.title('2018 heatmap')
sns.heatmap(df_industry_2018, annot=True, annot_kws = {"size": 8}, linewidth = 0.5, fmt='g', square=True, cmap = 'Reds_r')

fig.subplots_adjust(wspace=0.4)

plt.subplot(122)   #  subplot 2
plt.title('2021 heatmap')
sns.heatmap(df_industry_2021, annot=True, annot_kws = {"size": 8}, linewidth = 0.5, fmt='g', square=True, cmap = 'Blues_r')

plt.show()

Heatmap of occupation across different industries. Data scientist and software engineer names down notably compare to 2018.

In [33]:
# print(get_professionals(data_2018, 'Q6')['Q6'].unique())

pros_2018 = get_professionals(data_2018, 'Q6')
pros_2018 = pros_2018.loc[pros_2018['Q7'] != 'I am a student']
pros_2018 = pros_2018[pros_2018['Q7'].notna()]
pros_2018['Q6'].unique()

pros_2018['filter_Q6'] = pros_2018['Q6'].map(job_title)
pros_2018['filter_Q6'].value_counts()


In [34]:
professional_2021['filter_Q5'] = professional_2021['Q5'].map(job_title)
professional_2021['filter_Q5'].value_counts()

In [ ]:
student_2018 = data_2018.loc[data_2018['Q7'] == 'I am a student']
workforce_2018 = data_2018.loc[data_2018['Q7'] != 'I am a student']

professional_2018 = workforce_2018[workforce_2018['Q7'].notna()]
professional_2018.head()

industry_2018 = professional_2018['Q7'].unique()

df_2018 = professional_2018[['Q6','Q7']]

d_2018 = {}

for industry in industry_2018:
    temp_df = df_2018[df_2018['Q7'] == industry]
    temp_dict = dict(temp_df['Q6'].value_counts())
    d_2018[industry] = temp_dict

d_2018 = sorted_once_nested_dict(d_2018)
    
df_industry_2018 = pd.DataFrame.from_dict(d_2018, orient='index')
df_industry_2018.fillna(0, inplace = True)

In [3]:
import squarify
df = workforce_2021["Q5"].value_counts().rename_axis('Current role').reset_index(name='Counts')

plt.figure(figsize = (12,8))
squarify.plot(sizes = df['Counts'], label=df['Current role'], alpha=.8 )
plt.axis('off')

plt.show()


In [4]:
# get professionals only. It will drop student and not employed rows
# print(get_professionals(data_2021, 'Q5')['Q5'].unique())
# print(get_professionals(data_2020, 'Q5')['Q5'].unique())
# print(get_professionals(data_2019, 'Q5')['Q5'].unique())
# print(get_professionals(data_2018, 'Q6')['Q6'].unique())
# print(get_professionals(data_2017, 'CurrentJobTitleSelect')['CurrentJobTitleSelect'].unique())

def get_professionals(data, column):
    data = data.loc[data[column] != 'Student']
    data = data.loc[data[column] != 'Currently not employed']
    data = data.loc[data[column] != 'Not employed']
    data = data.loc[data[column].notna()]
    return data

test = get_professionals(data_2017, 'CurrentJobTitleSelect')
test["CurrentJobTitleSelect"].value_counts()
c = test["CurrentJobTitleSelect"].value_counts()
d = test["CurrentJobTitleSelect"].value_counts(normalize = True)
df = pd.concat([c, d], axis = 1, keys = ['counts', '%'])
df['%'].round(2)



In [5]:
test = get_professionals(data_2021, 'Q5')
test["Q5"].value_counts()
c = test["Q5"].value_counts()
d = test["Q5"].value_counts(normalize = True)
df = pd.concat([c, d], axis = 1, keys = ['counts', '%'])
df['%'].round(2)



In [6]:
test = get_professionals(data_2018, 'Q6')
test["Q6"].value_counts()
c = test["Q6"].value_counts()
d = test["Q6"].value_counts(normalize = True)
df = pd.concat([c, d], axis = 1, keys = ['counts', '%'])
df['%'].round(2)

In [7]:
print(get_professionals(data_2021, 'Q5')['Q5'].unique())
print(get_professionals(data_2020, 'Q5')['Q5'].unique())
print(get_professionals(data_2019, 'Q5')['Q5'].unique())
print(get_professionals(data_2018, 'Q6')['Q6'].unique())
print(get_professionals(data_2017, 'CurrentJobTitleSelect')['CurrentJobTitleSelect'].unique())

In [32]:
job_title = {'Other':'Other',
     'Product Manager': 'Product/Project Manager',
 'Program/Project Manager':'Product/Project Manager',
 'Principal Investigator':'Product/Project Manager',
 'Chief Officer':'Product/Project Manager',
 'Manager':'Product/Project Manager',
 'Software Developer/Software Engineer': 'Software Engineer',
 'Operations Research Practitioner': 'Research Scientist',
 'Computer Scientist': 'Research Scientist',
 'Scientist/Researcher': 'Research Scientist',
 'Researcher': 'Research Scientist',
 'Data Scientist': 'Data Scientist',
     'Business Analyst': 'Business Analyst',
     'Engineer': 'Other',
     'DBA/Database Engineer': 'DBA/Database Engineer',
     'Data Analyst':'Data Analyst',
     'Machine Learning Engineer': 'Machine Learning Engineer',
     'Statistician':'Statistician',
     'Predictive Modeler':'Research Scientist',
     'Programmer': 'Software Engineer',
     'Data Miner': 'Data Engineer',
     'Consultant': 'Other',
     'Research Assistant': 'Research Scientist',
     'Chief Officer':'Product/Project Manager',
     'Data Engineer':'Data Engineer',
     'Developer Advocate': 'Developer Relations/Advocacy',
     'Marketing Analyst': 'Business Analyst',
     'Data Analyst': 'Data Analyst',
     'Software Engineer': 'Software Engineer',
     'Research Scientist': 'Research Scientist',
     'Data Journalist': 'Data Analyst',
     'Salesperson':'Developer Relations/Advocacy',
     'Product/Project Manager': 'Product/Project Manager',
     'Developer Relations/Advocacy': 'Developer Relations/Advocacy'
}

#k = get_professionals(data_2017, 'CurrentJobTitleSelect')['CurrentJobTitleSelect'].unique()
k = get_professionals(data_2018, 'Q6')['Q6'].unique()
#k = get_professionals(data_2019, 'Q5')['Q5'].unique()
#k = get_professionals(data_2021, 'Q5')['Q5'].unique()
lstval = list(map(job_title.get, k))
set(lstval)
 

 

In [ ]:
jesus = data_2018.copy()
magic = get_professionals(jesus, 'Q6')
magic.replace({'Q6': job_title}, inplace = True)

In [ ]:
magic['Q6'].value_counts()

In [ ]:
Q11_Q12 = data_2021[['Q11', 'Q12_Part_1', 'Q12_Part_2', 'Q12_Part_3', 'Q12_Part_4', 'Q12_Part_5', 'Q12_OTHER']]

In [ ]:
Q11_Q12['Q11'].value_counts()

In [ ]:
for q in questions_2021:
    print(q)